# Comparing gaps 2011 to 2017

Tables required:

Math, grade 4, race eth
Math, grade 4, frpl  
Math, grade 8, race eth  
Math, grade 8, frpl  
Reading, grade 4, race eth  
Reading, grade 4, frpl  
Reading, grade 8, race eth  
Reading grade 8, frpl  

output file names:

    YYYY_subg_subj_grade.csv
   
   
output file columns

    state group1 group2 ... groupN postal ap_abbr
    

In [2]:
import pandas as pd

In [3]:
files = {
    "frpl":{
        "math":{
            "4":"2018-data/NDECoreExcel_Mathematics, Grade 4, National School Lunch Progra_20190102190215.Xls",
            "8":"2018-data/NDECoreExcel_Mathematics, Grade 8, National School Lunch Progra_20190102190714.Xls"
        },
        "reading":{
            "4":"2018-data/NDECoreExcel_Reading, Grade 4, National School Lunch Program el_20190102191340.Xls",
            "8":"2018-data/NDECoreExcel_Reading, Grade 8, National School Lunch Program el_20190102191038.Xls"
        }
    },
    "race":{
        "math":{
            "4":"2018-data/NDECoreExcel_Mathematics, Grade 4, Race ethnicity used to repor_20190102184218.Xls",
            "8":"2018-data/NDECoreExcel_Mathematics, Grade 8, Race ethnicity used to repor_20190102190614.Xls"
        },
        "reading":{
            "4":"2018-data/NDECoreExcel_Reading, Grade 4, Race ethnicity used to report tr_20190102191216.Xls",
            "8":"2018-data/NDECoreExcel_Reading, Grade 8, Race ethnicity used to report tr_20190102190845.Xls"
        }

    }
}

In [19]:
def get_df(subgroup, subject, grade):
    
    ret = pd.read_excel(
        files[subgroup][subject][str(grade)],
        skiprows=8
    )
    
    ret["Average scale score"] = pd.to_numeric(
        ret["Average scale score"],
        errors="coerce"
    )
    
    return ret

get_df("race", "reading", 4)

,Year,Jurisdiction,"Race/ethnicity used to report trends, school-reported",Average scale score
0,2017,National public,White,231.036997
1,2017,National public,Black,205.314165
2,2017,National public,Hispanic,208.027292
3,2017,National public,Asian/Pacific Islander,238.472596
4,2017,National public,American Indian/Alaska Native,202.813431
5,2017,National public,Two or more races,225.891673
6,2017,Alabama,White,227.695864
7,2017,Alabama,Black,201.956995
8,2017,Alabama,Hispanic,199.286096
9,2017,Alabama,Asian/Pacific Islander,NaN


In [108]:
def make_sheet(frame, year):
    
    ret = frame.copy()

    ret = ret[ret["Year"] == year]
    
    
#     groups = ret[ret.columns[2]].unique().tolist()
    
#     groups = map(lambda x: str(x).lower(), groups)
    
    group_col = ret.columns.unique().tolist()[2]
    
    ret = pd.pivot_table(
        ret,
        columns = group_col,
        index = "Jurisdiction",
        values = "Average scale score"
    )
    
    
    ret = ret.reset_index()
    ret.columns = ["state"] + map(lambda x: x.lower(), ret.columns.tolist())[1:]
    
    ret = ret.set_index("state")
    
    return ret

make_sheet(get_df("race", "reading", "4"), 2017).head().columns.tolist()

[u'american indian/alaska native',
 u'asian/pacific islander',
 u'black',
 u'hispanic',
 u'two or more races',
 u'white']

In [20]:
from us import states

In [87]:
ct = states.lookup(unicode("Connecticut"))
ct.ap_abbr

u'Conn.'

In [93]:
def abbr(state_name, func=lambda x: x.abbr, us="US"):
    if state_name == "National public": return us
    try:
        return func(states.lookup(unicode(state_name)))
    except Exception as e:
        print e
    
abbr("Connecticut")

u'CT'

In [96]:
ap_abbr = abbr

In [97]:
postal = lambda x: abbr(x, func=lambda x: x.ap_abbr, us="U.S.")
postal("Connecticut")

u'Conn.'

In [102]:
def fix_national(n):
    if n == "National public": return "Nation"
    return n


In [105]:
def make_file(subg, subj, grade, year):
    
    fname = "%s_%s_%s_%s.csv" % (str(subj), str(subg), str(grade), str(year))
    
    ret = make_sheet(get_df(subg, subj, grade), int(year))

    ret["postal"] = ret.index.map(postal)
    ret["ap_abbr"] = ret.index.map(ap_abbr)

    ret.index = ret.index.map(fix_national)
    
    ret.to_csv("viz-gaprank-2018/data/%s" % fname)
    
make_file("race","reading","4","2017")

'NoneType' object has no attribute 'ap_abbr'
'NoneType' object has no attribute 'abbr'


In [117]:
def make_combined_file(subj, grade, year):
    subgroups = "race", "frpl"
    fname = "2018_combined_%s_%s_%s.csv" % (str(subj), str(grade), str(year))
    print fname
    
make_combined_file("reading","4","2017")

2018_combined_reading_4_2017.csv


In [120]:
def make_combined_file(subj, grade, year):
    fname = "2018_combined_%s_%s_%s.csv" % (str(subj), str(grade), str(year))
    
    subgroups = {
        "race":[ u'american indian/alaska native',
                 u'asian/pacific islander',
                 u'black',
                 u'hispanic',
                 u'two or more races',
                 u'white'],
        "frpl":[
            "eligible",
            "not eligible"
        ]
    }
        
    
    ret = None
    for subg in ["race","frpl"]:
        print subg, subj, grade, year
        if ret is None:
            ret = make_sheet(get_df(subg, subj, grade), int(year))
        else:
            ret = ret.join(make_sheet(get_df(subg, subj, grade), int(year)))

    ret["postal"] = ret.index.map(postal)
    ret["ap_abbr"] = ret.index.map(ap_abbr)

    ret.index = ret.index.map(fix_national)
    
    ret.to_csv("viz-gaprank-2018/data/%s" % fname)

make_combined_file("reading","4","2017")

race reading 4 2017
frpl reading 4 2017
'NoneType' object has no attribute 'ap_abbr'
'NoneType' object has no attribute 'abbr'


In [106]:
for subg in ["race","frpl"]:
    for subj in ["math","reading"]:
        for year in [2017, 2011]:
            for grade in [4, 8]:
                make_file(subg, subj, grade, year)

'NoneType' object has no attribute 'ap_abbr'
'NoneType' object has no attribute 'abbr'
'NoneType' object has no attribute 'ap_abbr'
'NoneType' object has no attribute 'abbr'
'NoneType' object has no attribute 'ap_abbr'
'NoneType' object has no attribute 'abbr'
'NoneType' object has no attribute 'ap_abbr'
'NoneType' object has no attribute 'abbr'
'NoneType' object has no attribute 'ap_abbr'
'NoneType' object has no attribute 'abbr'
'NoneType' object has no attribute 'ap_abbr'
'NoneType' object has no attribute 'abbr'
'NoneType' object has no attribute 'ap_abbr'
'NoneType' object has no attribute 'abbr'
'NoneType' object has no attribute 'ap_abbr'
'NoneType' object has no attribute 'abbr'


In [121]:
# Make all combined files
for subj in ["math","reading"]:
    for year in [2017, 2011]:
        for grade in [4, 8]:
            make_combined_file(subj, grade, year)

race math 4 2017
frpl math 4 2017
'NoneType' object has no attribute 'ap_abbr'
'NoneType' object has no attribute 'abbr'
race math 8 2017
frpl math 8 2017
'NoneType' object has no attribute 'ap_abbr'
'NoneType' object has no attribute 'abbr'
race math 4 2011
frpl math 4 2011
'NoneType' object has no attribute 'ap_abbr'
'NoneType' object has no attribute 'abbr'
race math 8 2011
frpl math 8 2011
'NoneType' object has no attribute 'ap_abbr'
'NoneType' object has no attribute 'abbr'
race reading 4 2017
frpl reading 4 2017
'NoneType' object has no attribute 'ap_abbr'
'NoneType' object has no attribute 'abbr'
race reading 8 2017
frpl reading 8 2017
'NoneType' object has no attribute 'ap_abbr'
'NoneType' object has no attribute 'abbr'
race reading 4 2011
frpl reading 4 2011
'NoneType' object has no attribute 'ap_abbr'
'NoneType' object has no attribute 'abbr'
race reading 8 2011
frpl reading 8 2011
'NoneType' object has no attribute 'ap_abbr'
'NoneType' object has no attribute 'abbr'
